In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
df = pd.read_csv('../data/yob_ss/yob1880.txt', header=None,
                names = ['first', 'sex', 'count'])

#Per old actuarial methods I used, we'll be defining Male as anything not Female. Albeit even in 2023 all binary.
newborn_females, newborn_males = sum(df[df['sex']=='F']['count']), sum(df[df['sex']!='F']['count'])
df['per_of_year'] = [round(i/newborn_females, 3) if sex == 'F' else round(i/newborn_males, 3) for i, sex in zip(df['count'], df['sex'])]
male_name_start = df['sex'].value_counts()['F']
df['rank_of_year'] = [i if sex == 'F' else i - male_name_start for i, sex in zip(range(1, df.shape[0]+1), df['sex'])]
print(df.shape)

df.head()

#Hmm, potentally change the last two columns to make it clearer that it's gender specific? Oh well

(2000, 5)


,first,sex,count,per_of_year,rank_of_year
0,Mary,F,7065,0.078,1
1,Anna,F,2604,0.029,2
2,Emma,F,2003,0.022,3
3,Elizabeth,F,1939,0.021,4
4,Minnie,F,1746,0.019,5


Even if a name is androgynous, we're still intersted in seeing hte percentage of whatever gender being called it, to gauge name usage over time. Hence, depending upon the gender dividing a 'male' or 'female' name by the sum of all male/fe names used in whatever year.

In [69]:
#Hmm, but we have a gazillion... I see two approaches:
#Make a giant data frame and add all those guys to it, seperating them via year.

#2 Have a gazillion dataframes thenmaybe add as needed.

#Yeah, we're doing the former.

#Just realzied that we're assuming the SS lists are oganized by count. Perhaps we should be strict... oh well

birth_years = []

for specific_year in range(1880, 2023):
    curr_df = pd.read_csv(f'../data/yob_ss/yob{specific_year}.txt', header=None,
                names = ['first', 'sex', 'count'])
    curr_df['year'] = specific_year
    newborn_females, newborn_males = sum(curr_df[curr_df['sex']=='F']['count']), sum(curr_df[curr_df['sex']!='F']['count'])
    curr_df['per_of_year'] = [round(i/newborn_females, 3) if sex == 'F' else round(i/newborn_males, 3) for i, sex in zip(curr_df['count'], curr_df['sex'])]
    male_name_start = curr_df['sex'].value_counts()['F']
    curr_df['rank_of_year'] = [i if sex == 'F' else i - male_name_start for i, sex in zip(range(1, curr_df.shape[0]+1), curr_df['sex'])]
    birth_years.append(curr_df)
    
df = pd.concat(birth_years, axis=0, ignore_index=True) #Apparently it's faster to do it this way. More intuitive to be to
#update the 'final' df immediately, but at least upon asking ChatGPT they claim it's quicker this way. Although we only have
#4 features, including year, we do have over 2m records.
    
print(df.shape)
df.head()

(2085537, 6)


,first,sex,count,year,per_of_year,rank_of_year
0,Mary,F,7065,1880,0.078,1
1,Anna,F,2604,1880,0.029,2
2,Emma,F,2003,1880,0.022,3
3,Elizabeth,F,1939,1880,0.021,4
4,Minnie,F,1746,1880,0.019,5


Hmm, I'm wondering if SQL might have been the way to go for this. Eh, oh well.

In [48]:
df[(df['sex'] == 'M') & (df['year'] < 1900)].sort_values('count', ascending=False)[:10]

,first,sex,count,year,per_of_year
942,John,M,9655,1880,0.087
4962,John,M,9557,1882,0.084
943,William,M,9532,1880,0.086
9317,John,M,9388,1884,0.082
4963,William,M,9298,1882,0.082
18975,John,M,9247,1888,0.077
29758,John,M,9039,1892,0.074
14018,John,M,9026,1886,0.081
9318,William,M,8897,1884,0.078
7115,John,M,8894,1883,0.085


In [36]:
df['sex'].value_counts() #Wow.... and who said it's 50/50?!!?!? Not even close.... perhaps in one year is different
#Oh no noob - this refers to just names... we'd need to sum the counts...

#Still interesting though... albeit many are duplicated

sex
F    1228026
M     857511
Name: count, dtype: int64

In [44]:
len(df[df['sex']=='F']['first'].unique()), len(df[df['sex']!='F']['first'].unique()) #There we go. Hmm, a similar disparity,
#which somewhat makes sense.Regardless, see the much greater varity of female names than male names. Now, recall we're dealing
#with the United States - a country that has many immigrants coming in, more and more as time goes on.So, a bit harder to gauge
#the proportion of names when certain cultures bring with it differnt proportions of the gendered names.

(70244, 43678)

In [32]:
#Now, this is what I meant to do:

females, males = sum(df[df['sex']=='F']['count']), sum(df[df['sex']!='F']['count'])
females,males
#Hmm, so about 4 million more males born over 144 years. Interesting. Might be neat to do a thing where we log
#the number of males/females born each year from 1880 to 2023, potetialy focusing on percentages to cancel the effect of population
#growth.

(180703184, 184635808)

In [46]:
round(females/len(df[df['sex']=='F']['first'].unique()), 3), round(males/len(df[df['sex']!='F']['first'].unique()), 3)
#Wow, the average female name is used a bit more than half of the average male name. Obviously from just our head we see Mary and Anna
#beat that - just with that one year... but we're talking about all 70k unique female names...

(2572.507, 4227.204)

In [38]:
df[df['year'] == 1880]['sex'].value_counts()

sex
M    1058
F     942
Name: count, dtype: int64

## Select Analytics and Visuals

Hmm, what should we focus on? I'll at least give 3 ideas right now:

Note, by default assume things are gendered (do it both for male and female)

Most popular name from 1880 useage until now.
    From 2023 and how it changes (if it did)
Occurences of names in the top 10, 20, 25, 30, 50, 100

Gendered births over time (ie did begetting males become less frequent?)